In [15]:
import openai
import os
from dotenv import load_dotenv

# --- Configuration ---
# Replace with your actual OpenAI API key
# It's best to set this as an environment variable for security
# For example: openai.api_key = os.getenv("OPENAI_API_KEY")
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

client = openai.OpenAI()

explanation_text = None
try:
    print(f"\n🤖 Text wird generiert...")
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",  # Or your preferred model like "gpt-4o"
        messages=[
            {"role": "system", "content": "Du bist ein nützlicher und hilfsbereiter Assisstent, welcher klare und simple Erklärungen liefert."},
            {"role": "user", "content": f"Philosophiere über das Leben und die Reize des Meeres."}
        ],
        max_tokens=150,
        temperature=0.7
    )
    explanation_text = response.choices[0].message.content.strip()
    print(f"\n📜 Explanation:\n{explanation_text}")
except Exception as e:
    print(f"Error generating text with OpenAI: {e}")


🤖 Generating explanation for 'What is the meaning of life?'...

📜 Explanation:
Das Leben und das Meer haben viele Parallelen, da beide unberechenbar und voller Schönheit sind. Das Leben, wie das Meer, kann ruhig und friedlich sein, aber auch stürmisch und gefährlich. Die Wellen des Meeres symbolisieren die Höhen und Tiefen, die wir im Leben erleben. Die Weite des Meeres erinnert uns daran, wie groß und vielfältig die Welt ist, und ermutigt uns, nach neuen Abenteuern zu streben.

Die Reize des Meeres sind unendlich: Der Klang der Wellen, der Geruch des Salzwassers, die Weite des Horizonts


In [29]:
import playsound
from pathlib import Path

if explanation_text:
    print("\n🔊 Converting text to OpenAI speech...")
    cur_dir = os.getcwd()
    speech_file_path = cur_dir + "/speech.mp3"

    try:
        # Use .with_streaming_response for the API call
        with client.audio.speech.with_streaming_response.create(
            model="tts-1",
            voice="alloy",
            input=explanation_text,
            # response_format="mp3" # mp3 is default
        ) as response:
            # Now, use the .stream_to_file() method from the streaming response object
            response.stream_to_file(speech_file_path)

        print(f"Audio saved as {speech_file_path}")
        print("Playing audio...")
        playsound.playsound(str(speech_file_path))
        print("Playback finished.")

    except Exception as e:
        print(f"Error with OpenAI TTS or playing sound: {e}")
        if "401" in str(e): # Basic check for authentication error
             print("This might be an issue with your OpenAI API key or organization setup for TTS.")
        elif "Unsupported olha" in str(e) or "Unsupported model" in str(e): # Example for model/voice error
             print("Please ensure the selected voice and model for TTS are supported by your API access.")
else:
    print("No explanation text to convert to speech.")


🔊 Converting text to OpenAI speech...
Audio saved as /home/philipp/projects/gruene/podcast/speech.mp3
Playing audio...
Playback finished.
